In [1]:
# Imports

import numpy as np
import pandas as pd
from scipy import stats
import sklearn.preprocessing as p
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")
import env
import prepare_regression_2 as pr
import wrangle_2 as w
# import explore_2 as e

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import TweedieRegressor

%matplotlib inline

Do your work for this exercise in a jupyter notebook named modeling within the regression-exercises repo. Add, commit, and push your work.

Select a dataset with a continuous target variable.

In [2]:
def sql_zillow_data():
    sql_query = """
                select prop.parcelid as parcel_id, prop.id as property_id, prop.bathroomcnt, prop.bedroomcnt, prop.regionidzip, prop.yearbuilt, prop.calculatedfinishedsquarefeet, prop.fips, prop.taxvaluedollarcnt
                from predictions_2017 as pred
                join properties_2017 as prop 
                on pred.parcelid = prop.parcelid
                where transactiondate is not null and
                propertylandusetypeid = 261 and
                regionidzip is not null and
                yearbuilt is not null and
                calculatedfinishedsquarefeet is not null and
                prop.taxvaluedollarcnt is not null;
                """
    df = pd.read_sql(sql_query, env.get_connection('zillow'))
    return df


In [3]:
df = sql_zillow_data()

In [4]:
def prep_zillow(df):

    df = df.rename(columns={'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms',
                          'yearbuilt':'year_built',
                          'regionidzip': 'zip_code',
                          'calculatedfinishedsquarefeet':'total_sqft',
                          'fips': 'county',
                          'taxvaluedollarcnt': 'tax_value'
                          })

    df = df[df.bathrooms <= 8]
    df = df[df.bathrooms >= 1]
    df = df[df.bedrooms <= 8]
    df = df[df.bedrooms >= 2]
    df = df[df.zip_code < 399675]
    df.zip_code = df.zip_code.convert_dtypes(int)
    df.total_sqft = df.total_sqft.convert_dtypes(int)
    df.bedrooms = df.bedrooms.convert_dtypes(int)
    df.year_built = df.year_built.convert_dtypes(int)
    df.county = df.county.replace(6059.0,'Orange').replace(6037.0,'Los_Angeles').replace(6111.0,'Ventura')
    

    #df = df[df.sqft < 10000]

    #df = df.dropna()
    df.drop_duplicates(inplace=True)
    
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
    
    return train, validate, test


In [5]:
train, validate, test = prep_zillow(df)

In [6]:
train.shape, validate.shape, test.shape

((28817, 9), (12351, 9), (10292, 9))

Be sure your data is prepared (no missing values, numeric datatypes) and split into samples.

Work through all of the steps outlined in the lesson, from setting the baseline to selected a model and evaluating the final model on your test data.

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28817 entries, 31793 to 25682
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   parcel_id    28817 non-null  int64  
 1   property_id  28817 non-null  int64  
 2   bathrooms    28817 non-null  float64
 3   bedrooms     28817 non-null  Int64  
 4   zip_code     28817 non-null  Int64  
 5   year_built   28817 non-null  Int64  
 6   total_sqft   28817 non-null  Int64  
 7   county       28817 non-null  object 
 8   tax_value    28817 non-null  float64
dtypes: Int64(4), float64(2), int64(2), object(1)
memory usage: 2.3+ MB


In [8]:
train.isna().count()

parcel_id      28817
property_id    28817
bathrooms      28817
bedrooms       28817
zip_code       28817
year_built     28817
total_sqft     28817
county         28817
tax_value      28817
dtype: int64

In [9]:
def county_dummies_all(train_1, validate_1, test_1):
    train_1, validate_1, test_1 = prep_zillow(df)
    train_1_encoded = pd.get_dummies(train_1['county'], drop_first=False)  
    train_1_encoded = train_1.merge(train_1_encoded, left_index=True, right_index=True)
    train_1_encoded = train_1_encoded.drop(columns= 'county')

    validate_1_encoded = pd.get_dummies(validate_1['county'], drop_first=False)  
    validate_1_encoded = validate_1.merge(validate_1_encoded, left_index=True, right_index=True)
    validate_1_encoded = validate_1_encoded.drop(columns= 'county')

    test_1_encoded = pd.get_dummies(test_1['county'], drop_first=False)  
    test_1_encoded = test_1.merge(test_1_encoded, left_index=True, right_index=True)
    test_1_encoded = test_1_encoded.drop(columns= 'county')

    return train_1_encoded, validate_1_encoded, test_1_encoded

In [10]:
def model_prep(df1, df2, df3):
    df1, df2, df3 = county_dummies_all(df1, df2, df3)
    df1 = df1.drop(columns= ['parcel_id', 'property_id', 'zip_code'])
    df2 = df2.drop(columns= ['parcel_id', 'property_id', 'zip_code'])
    df3 = df3.drop(columns= ['parcel_id', 'property_id', 'zip_code'])
    return df1, df2, df3

In [11]:
def X_train_y_train_split(df):
    X_train = df.drop(columns = 'tax_value')
    y_train = df.drop(columns = ['bathrooms' , 'bedrooms' , 'year_built' , 'total_sqft' , 'Los_Angeles' , 'Orange' , 'Ventura'])
    return X_train, y_train

In [12]:
train, validate, test = model_prep(train, validate, test)

X_train, y_train = X_train_y_train_split(train)
X_validate, y_validate = X_train_y_train_split(validate)
X_test, y_test = X_train_y_train_split(test)

In [13]:
X_train.shape

(28817, 7)

In [14]:
X_train.head()

,bathrooms,bedrooms,year_built,total_sqft,Los_Angeles,Orange,Ventura
31793,3.0,4,1978,2322,1,0,0
52132,3.0,4,2007,3537,1,0,0
36759,2.0,4,1964,1326,1,0,0
27845,3.0,4,1937,1774,1,0,0
10940,2.0,4,1953,1795,0,1,0


Now scale the data

In [15]:
#minmaxscaler
def scale_zillow(train, validate, test):

    train_1 = train.copy()
    validate_1 = validate.copy()
    test_1 = test.copy()

    scale_cols = ['bedrooms', 'bathrooms', 'total_sqft', 'year_built']
    minmax_scaler = p.MinMaxScaler()
    minmax_scaler.fit(train_1[scale_cols])

    train_1[scale_cols] = minmax_scaler.transform(train[scale_cols])
    validate_1[scale_cols] = minmax_scaler.transform(validate[scale_cols])
    test_1[scale_cols] = minmax_scaler.transform(test[scale_cols])

    df_train_1 = pd.DataFrame(train_1).set_index([train_1.index.values])
    df_validate_1 = pd.DataFrame(validate_1).set_index([validate_1.index.values])
    df_test_1 = pd.DataFrame(test_1).set_index([test_1.index.values])

    return df_train_1, df_validate_1, df_test_1

In [16]:
#setting baseline variables

y_train['value_pred_mean'] = 527866.30
y_validate['value_pred_mean'] = 527866.30

y_train['value_pred_median'] = 376866.00
y_validate['value_pred_median'] = 376866.00

In [17]:
#defining root mean square error
def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

In [18]:
def GLM(power, alpha):
    # create the model object
    glm = TweedieRegressor(power=power, alpha=alpha)

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    glm.fit(X_train, y_train.tax_value)

    # predict train
    y_train['value_pred_lm'] = glm.predict(X_train)

    # evaluate: rmse
    rmse_train = rmse(y_train.tax_value, y_train.value_pred_mean)

    # predict validate
    y_validate['value_pred_lm'] = glm.predict(X_validate)

    # evaluate: rmse
    rmse_validate = rmse(y_validate.tax_value, y_validate.value_pred_median)

    return print("RMSE for GLM using TweedieRegressor\nTraining/In-Sample: ", round(rmse_train), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate))


    

In [19]:
GLM(1,0)

RMSE for GLM using TweedieRegressor
Training/In-Sample:  712593 
Validation/Out-of-Sample:  650380
